In [1]:
pip install -r ../../requirements.txt


[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [3]:
from zkstats.core import prover_gen_settings, verifier_setup, prover_gen_proof, verifier_verify

In [4]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
# srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

## Step 1
Verifier calls `export_onnx` with dummy data, to generate the onnx model. Dummy data is used to infer the shape of the model

### Output
- `verifier.onnx` file

In [5]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

## Step 2
- User defines their computation in a function with signature `computation(state: State, x: list[torch.Tensor])`.
- Prover calls `create_model(computation)` to derive the actual model.
- Prover calls `prover_gen_settings`: export onnx file and compute the settings required by `ezkl.calibrate_settings`

In [6]:
from zkstats.computation import State, create_model


def computation(state: State, x: list[torch.Tensor]):
    print("!@# x:", x)
    out_0 = state.median(x[0])
    out_1 = state.median(x[0])
    return state.mean(torch.tensor([out_0, out_1]).reshape(1,-1,1))

prover_model = create_model(computation)
prover_gen_settings([data_path], comb_data_path, prover_model, prover_model_path, "default", "resources", settings_path)


!@# x: [tensor([[[23.2000],
         [92.8000],
         [91.0000],
         [37.2000],
         [82.0000],
         [15.5000],
         [79.3000],
         [46.6000],
         [98.1000],
         [75.5000],
         [78.9000],
         [77.6000],
         [33.8000],
         [75.7000],
         [96.8000],
         [12.3000],
         [18.4000],
         [13.4000],
         [ 6.0000],
         [ 8.2000],
         [25.8000],
         [41.3000],
         [68.5000],
         [15.2000],
         [74.7000],
         [72.7000],
         [18.0000],
         [42.2000],
         [36.1000],
         [76.7000],
         [ 1.2000],
         [96.4000],
         [ 4.9000],
         [92.0000],
         [12.8000],
         [28.2000],
         [61.8000],
         [56.9000],
         [44.3000],
         [50.4000],
         [81.6000],
         [72.5000],
         [12.9000],
         [40.3000],
         [12.8000],
         [28.8000],
         [36.3000],
         [16.1000],
         [68.4000],
         [35

IndexError: list index out of range

## Step 3
- Prover generates proof with `prover_gen_proof`:
    - 1. Compile circuit with `compile_circuit`
        - Inputs: onnx model, settings path
        - Outputs: compiled 
    - 2. Generate witness with
- Verifier generates keys with 

In [ ]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(prover_model_path, prover_compiled_model_path, settings_path, vk_path, pk_path )

print("=======================================")
# Prover generates proof
# print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

In [ ]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path)